In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('..')
# load 
from train import LatentTranslatorTrainer
# %%
trainer = LatentTranslatorTrainer.load_from_checkpoint('../models/latent_translator/dauntless-resonance-23/epoch=0-step=29795.ckpt')
# %%
trainer.model.bfloat16()
trainer.model.cuda()
trainer.model.eval()
# %%

In [3]:
import torch

vid = torch.load('/home/developer/workspace/data/tensors/valid/0000.pt')

from lat2lat.models.wan_dcae import WANDCAEPair

pairencoder = WANDCAEPair()

In [ ]:
vid.shape
pairencoder.bfloat16()
pairencoder.cuda()
pairencoder.eval()


In [ ]:
a, b = pairencoder.preprocess(vid[21:21+21].unsqueeze(0))
with torch.no_grad():
    za,zb = pairencoder(a.cuda(),b.cuda())
print(za.shape, zb.shape)

In [13]:
za_ = trainer.model.translate_batch(zb.cuda())

In [14]:
# load wan decoder 
import sys
sys.path.append('../owl-vaes')
sys.path.append('../')
import torch
from diffusers import AutoencoderKLWan

# Load the VAE
vae = AutoencoderKLWan.from_pretrained(
    "Wan-AI/Wan2.1-VACE-14B-diffusers",
    subfolder="vae",
    torch_dtype=torch.bfloat16  # Optional: use half precision
)

# Move to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
vae = vae.to(device)

In [ ]:
# make a boxplot of za_ 
from matplotlib import pyplot as plt

plt.boxplot(za_.detach().cpu().float().flatten().numpy())
plt.show()

In [ ]:
from lat2lat.utils import create_video_visualization, export_video_as_gif
from IPython.display import display, HTML
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np

with torch.no_grad():
    zprep = za_.permute(0, 2, 1,3,4)
    recon_vid = vae.decode(zprep.to(vae.device))
    recon_vid = recon_vid.sample.permute(0, 2, 1,3,4)[0]
    
    # Also decode the original za for comparison
    za_prep = za.permute(0, 2, 1,3,4)
    za_vid = vae.decode(za_prep.to(vae.device))
    za_vid = za_vid.sample.permute(0, 2, 1,3,4)[0]

# Example usage with your reconstructed video
print("Creating video visualizations...")

# 1. Animated playback - side by side comparison with synchronized playback
print("\n1. Creating synchronized side-by-side video comparison...")

# Create figure with two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
fig.suptitle('Synchronized Video Comparison', fontsize=16)

# Get video data
_, video_np_za = create_video_visualization(za_vid, "Original Video (za)")
_, video_np_recon = create_video_visualization(recon_vid, "Reconstructed Video (za_)")

# Ensure both videos have the same number of frames
min_frames = min(len(video_np_za), len(video_np_recon))
video_np_za = video_np_za[:min_frames]
video_np_recon = video_np_recon[:min_frames]

# Initialize plots
im1 = ax1.imshow(video_np_za[0])
ax1.set_title('Original Video (za)')
ax1.axis('off')

im2 = ax2.imshow(video_np_recon[0])
ax2.set_title('Reconstructed Video (za_)')
ax2.axis('off')

def animate(frame):
    im1.set_array(video_np_za[frame])
    im2.set_array(video_np_recon[frame])
    return [im1, im2]

# Create animation with synchronized playback
anim = animation.FuncAnimation(
    fig, animate, frames=min_frames, 
    interval=100, blit=True, repeat=True
)

# Display the synchronized animation
display(HTML(anim.to_jshtml()))